In [7]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
import os
from dotenv import load_dotenv

# Load .env
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")

if not API_KEY:
    raise ValueError("❌ GOOGLE_API_KEY not found in .env file")

# Create agent
news_analysis_agent = LlmAgent(
    name="news_analysis_agent",
    model=Gemini(
        model="gemini-2.0-flash-001",
        api_key=API_KEY,          # <── REQUIRED FIX
        generation_config={
            "temperature": 0.15,
            "top_p": 0.95,
            "max_output_tokens": 1500,
        }
    ),
    instruction="""You are a Financial News Scraper.
            1. Use google_search tool to find the latest news articles about the given stock ticker.
            2. Analyze the overall sentiment (Bullish/Bearish/Neutral) based on the articles.
            3. Return ONLY valid JSON with no markdown formatting.
            
            Required JSON format:
            {
              "overall_sentiment": "Bullish",
              "overall_sentiment_score": 0.45,
              "final_conclusion": "Brief summary of market sentiment based on news.",
              "articles": [
                {"headline": "Article title here", "source": "Source name"},
                {"headline": "Another article", "source": "Another source"}
              ]
            }
            
            Make sure to include at least 3-5 articles if available.
            """,
    tools=[google_search],
    output_key="news-analysis-report"
)

# Runner
news = InMemoryRunner(agent=news_analysis_agent)

print("✅ news_analysis_agent created successfully!")


✅ news_analysis_agent created successfully!


In [8]:
response = await news.run_debug(
    "NCC Ltd "
)


 ### Created new session: debug_session_id

User > NCC Ltd 
news_analysis_agent > ```json
{
  "overall_sentiment": "Bearish",
  "overall_sentiment_score": -0.6,
  "final_conclusion": "NCC Ltd. is currently facing a challenging market environment. While the company has a strong order book and has secured new orders, it is also dealing with declining financial performance, increased debt levels, and is trading in an oversold zone. Technical analysis suggests a bearish trend, and analysts advise caution. Therefore, the sentiment is predominantly bearish.",
  "articles": [
    {"headline": "NCC Sees Significant Open Interest Surge Amidst Mixed Market Signals", "source": "MarketsMojo"},
    {"headline": "NCC gains 4% on ₹2,063 cr Gauhati Medical College expansion order", "source": "Business Standard"},
    {"headline": "NCC share up 3% on securing orders worth ₹6,829 cr from Central Coalfields", "source": "Business Standard"},
    {"headline": "This Jhunjhunwala stock is stuck in bear grip

In [4]:
"""News Analysis Module using Google Gemini"""
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
import os
from dotenv import load_dotenv

# Load .env
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")

if not API_KEY:
    raise ValueError("❌ GOOGLE_API_KEY not found in .env file")


async def get_news_data_async(symbol: str):
    """Fetch news using Gemini with clean output."""
    try:
        news_analysis_agent = LlmAgent(
        name="news_analysis_agent",
        model=Gemini(
            model="gemini-2.0-flash-001",
            api_key=API_KEY,          # <── REQUIRED FIX
            generation_config={
                "temperature": 0.15,
                "top_p": 0.95,
                "max_output_tokens": 1500,
            }
        ),
        instruction="""You are a Financial News Scraper.
                1. Use google_search tool to find the latest news articles about the given stock ticker.
                2. Analyze the overall sentiment (Bullish/Bearish/Neutral) based on the articles.
                3. Return ONLY valid JSON with no markdown formatting.
                
                Required JSON format:
                {
                "overall_sentiment": "Bullish",
                "overall_sentiment_score": 0.45,
                "final_conclusion": "Brief summary of market sentiment based on news.",
                "articles": [
                    {"headline": "Article title here", "source": "Source name"},
                    {"headline": "Another article", "source": "Another source"}
                ]
                }
                
                Make sure to include at least 3-5 articles if available.
                """,
        tools=[google_search],
        output_key="news-analysis-report"
    )

        # Runner
        runner = InMemoryRunner(agent=news_analysis_agent)

        
        with contextlib.redirect_stdout(io.StringIO()), contextlib.redirect_stderr(io.StringIO()):
            result = await runner.run_debug(f"Search for latest news about {symbol} stock and analyze sentiment. Return JSON report with articles.")
        
        text_output = ""
        
        if isinstance(result, list) and len(result) > 0:
            last_turn = result[-1]
            
            if hasattr(last_turn, 'output'):
                if hasattr(last_turn.output, 'text'):
                    text_output = last_turn.output.text
                elif hasattr(last_turn.output, 'content'):
                    text_output = str(last_turn.output.content)
            elif hasattr(last_turn, 'text'):
                text_output = last_turn.text
            elif hasattr(last_turn, 'content'):
                text_output = str(last_turn.content)
            else:
                text_output = str(last_turn)
        
        elif hasattr(result, 'output'):
            if hasattr(result.output, 'text'):
                text_output = result.output.text
            else:
                text_output = str(result.output)
        elif hasattr(result, 'text'):
            text_output = result.text
        else:
            text_output = str(result)

        text_output = re.sub(r'```json\s*', '', text_output, flags=re.IGNORECASE)
        text_output = re.sub(r'```\s*', '', text_output)
        text_output = text_output.strip()
        
        json_match = re.search(r'\{[\s\S]*"articles"[\s\S]*\}', text_output)
        if json_match:
            json_str = json_match.group(0)
            try:
                parsed = json.loads(json_str)
                return parsed
            except json.JSONDecodeError:
                pass
        
        try:
            parsed = json.loads(text_output)
            return parsed
        except:
            pass
        
        return {
            "overall_sentiment": "Neutral",
            "overall_sentiment_score": 0,
            "final_conclusion": "Unable to fetch news data at this time.",
            "articles": []
        }

    except Exception as e:
        return {
            "overall_sentiment": "Neutral",
            "overall_sentiment_score": 0,
            "final_conclusion": f"Error fetching news: Unable to retrieve current news data.",
            "articles": []
        }

In [5]:
await get_news_data_async("NCC.NS")

{'overall_sentiment': 'Bullish',
 'overall_sentiment_score': 0.6,
 'final_conclusion': 'The overall sentiment for NCC.NS is bullish, driven by strong buy recommendations from analysts, positive revenue forecasts, and a large order book. However, investors should be aware of declining financial performance and increased debt levels.',
 'articles': [{'headline': 'NCC Share Price Today 26 Nov 2025: Live NSE/BSE Rates, Technical Analysis and Expert Forecasts',
   'source': 'Mint'},
  {'headline': 'NCC.NS Stock Forecast & Price Target',
   'source': 'vertexaisearch.cloud.google.com'},
  {'headline': 'NCC Share Price: NCC Live Chart & News - Tickertape',
   'source': 'Tickertape'},
  {'headline': 'NCC Ltd Share Price Today - Live NSE/BSE Rate & Chart',
   'source': 'Motilal Oswal'},
  {'headline': 'NCC share price today - Live NSE/BSE | The Economic Times',
   'source': 'The Economic Times'}]}